In [1]:
import pandas as pd

In [ ]:
clicks["ref_type"].value_counts()

QUE ES CADA COLUMNA, formato, y si esta presente en otro DF (con mismo o distinto nombre)

AUCTIONS
auction_type_id: todo NaN, DESCARTADA
country: todos mismo pais, DESCARTADA
date: fecha
device_id: ID del dispositivo. es equivalente a ref_hash de clicks e installs
platform: plataforma. tiene 2 valores posibles asi que debe ser ios y android
ref_type_id: ¿?  tiene MISMOS 2 valores posibles que platform
source_id:  5 valores posibles, ni idea que es
    

    
CLICKS
advertiser_id: 7 valores posibles. hay un unico que es la mayoria de las veces
action_id: todo NaN, DESCARTADA
source_id: 11 valores posibles. ni idea
created: fecha creacion
country_code: todos mismo pais, DESCARTADA
latitude: latitud
longitude: longitud 
wifi_connection: todos false. DESCARTADA
carrier_id: ni idea que es carrier. tiene varios valores posibles
trans_id: supongo que id de transaccion? son todos unicos
os_minor: ni idea,  varios valores posibles
agent_device: ???
os_major: 10 valores posibles, ni idea
specs_brand: 5 valores posibles. 2 con muchisima cantidad y 3 con re pocas veces. marca de algo?
brand: marca de algo?
timeToClick: ¿tiempo que tarda en clickear desde que aparece? en segundos
touchX:  ¿posicion de la pantalla en el eje horizontal?
touchY:  ¿posicion de la pantalla en el eje vertical?
ref_type: 4 valores posibles. ni idea
ref_hash: id del dispositivo. equivalente a device id de auctions y ref hash de installs
    
    
EVENTS
date: date
event_id: id del evento
ref_type:  apple_ifa o google_advertising_id. supongo que el que tiene mas cantidad es google
ref_hash: no se que es pero muchos valores se repiten. algunos mas de 2000 veces. varios individuales tamb
application_id: id de app supongo? la explicacion en el drive es rara
attributed: cuantas subastas gano jampp. booleano
device_countrycode: todos repetidos. descartar
device_os_version: version del sistema operativo? hay muchos y muchos repetidos
device_brand: marca del dispositivo? medio raro porque hay muchas marcas que tienen solo 1 registro
device_model: modelo del celular, varios repetidos
device_city: ciudad, varios repetidos
session_user_agent: hash de la sesion?
trans_id:
user_agent: 
event_uuid:
carrier: suponemos que es empresa de telefonia pero no tieen mucho sentid oa cantidad de valores posibles
kind:
device_os: sistema operativo del dispositivo
wifi: si estan con wifi o no
connection_type: si esta por cable o por antena de 4g
ip_address: ip? se puede agrupar. raro que algunos se repitan mucho
device_language:
    

    
INSTALLS
created: -
application_id:
ref_type:
ref_hash:
click_hash:
attributed:
implicit:
device_countrycode:
device_brand:
device_model:
session_user_agent:
user_agent:
event_uuid:
kind:
wifi:
trans_id:
ip_address:
device_language:


In [36]:
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'matplotlib'

In [34]:
events["ip_address"].value_counts().plot()


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
auctions = pd.read_csv('auctions.csv')

In [3]:
clicks = pd.read_csv('clicks.csv')

In [4]:
events = pd.read_csv('events.csv')

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
installs = pd.read_csv('installs.csv')

In [ ]:
auctions.head()

In [ ]:
clicks.head()

In [ ]:
events.head()

In [ ]:
events["connection_type"].value_counts()

In [ ]:
installs.head()

In [ ]:
# Un solo pais
auctions["country"].unique()

In [ ]:
# INUTIL
clicks['wifi_connection'].unique()

In [ ]:
# UTIL
installs['wifi'].value_counts()

In [ ]:
installs.columns

In [ ]:
list(installs.columns)

In [15]:
installs_columns = list(events.columns)
installs_columns.remove("date")
for i in installs_columns:
    print(i," ",events[i].value_counts())
    
# Importante en 

event_id   22     618228
31     417070
161    217846
1      157812
138    100039
162     84898
329     56490
156     53332
33      48581
171     43942
310     42101
155     38465
273     37056
23      35588
21      26228
98      24301
158     23738
272     22788
80      22076
50      20905
81      18622
346     14408
58      13940
51      13455
513     12682
354     11711
133     11074
256     10939
274     10035
60       9456
        ...  
500         1
511         1
532         1
516         1
132         1
520         1
521         1
523         1
575         1
577         1
578         1
579         1
608         1
433         1
597         1
436         1
589         1
588         1
587         1
586         1
585         1
140         1
583         1
581         1
580         1
437         1
443         1
227         1
226         1
584         1
Name: event_id, Length: 568, dtype: int64
ref_type   1891515180541284343    1882743
1494519392962156891     611680
Name: ref_type, dtyp

session_user_agent   3.819516e+18    1461049
7.164321e+18     649962
1.160952e+18      78232
2.394269e+18      11210
3.831619e+18      10838
6.593868e+18       9911
2.495805e+18       7391
7.512089e+18       6995
7.738334e+18       6770
7.720270e+18       6626
5.254247e+18       6450
4.392265e+18       5998
9.177081e+18       5615
1.244062e+18       5525
3.590748e+18       5448
6.840860e+18       5055
1.637986e+18       4872
5.407282e+18       4619
5.519609e+18       3914
4.525727e+18       3724
4.472077e+18       3657
4.185912e+18       3436
6.105227e+18       3412
8.089178e+18       3326
2.955494e+18       3267
4.259154e+18       3194
4.455209e+18       3008
6.368947e+18       3004
6.091499e+18       2755
8.227345e+18       2496
                 ...   
3.100191e+18          1
6.927155e+18          1
5.527631e+15          1
2.563631e+18          1
7.114784e+18          1
5.730547e+18          1
6.044217e+18          1
4.332253e+18          1
8.112828e+18          1
9.024392e+18       

wifi   True     930902
False    447970
Name: wifi, dtype: int64
connection_type   Cable/DSL    331948
Cellular     280511
Corporate         4
Name: connection_type, dtype: int64
ip_address   1992632945768888579    22640
3162368506662488065    19379
7726314782926052608    17704
2394721645995561079     9777
589886713227299026      7561
7833422721300884983     7197
3564864895681659759     7142
4537476465028897433     7140
2403839579048525602     7100
7410847412569818934     7065
8235954648733428120     6933
7481987231813232849     6855
1093212093074216349     6347
8010533080341024188     6241
2441645233986204328     6108
2706357072127963511     4993
8262991154202232888     4736
1037699812701179191     4554
2988221197536288219     4408
2322330997698851930     3901
6286868732416664667     3698
8928734162578817363     3519
8531002764348621869     2710
7326588564616977029     2595
6452955265823183819     2571
7571354015197421297     2556
7858558567428669000     2520
3222899837426106171     25

In [12]:
installs_columns = list(installs.columns)
installs_columns.remove("created")
for i in installs_columns:
    print(i," ",installs[i].value_counts())
    
# Importante en 

ValueError: list.remove(x): x not in list

In [25]:
events_columns = list(events.columns)
print events_columns
events_columns.remove('date')
for i in events_columns:
    print events[i].value_counts()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(events_columns)? (<ipython-input-25-04bea739c9b3>, line 2)

In [26]:
installs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 18 columns):
created               3412 non-null object
application_id        3412 non-null int64
ref_type              3412 non-null int64
ref_hash              3412 non-null int64
click_hash            0 non-null float64
attributed            3412 non-null bool
implicit              3412 non-null bool
device_countrycode    3412 non-null int64
device_brand          1047 non-null float64
device_model          3411 non-null float64
session_user_agent    3364 non-null object
user_agent            1729 non-null object
event_uuid            865 non-null object
kind                  865 non-null object
wifi                  1729 non-null object
trans_id              6 non-null object
ip_address            3412 non-null int64
device_language       3378 non-null float64
dtypes: bool(2), float64(4), int64(5), object(7)
memory usage: 433.2+ KB


In [27]:
installs["session_user_agent"].value_counts()

http-kit/2.0                                                                                                                                                                                                                                                                  1729
adjust.com                                                                                                                                                                                                                                                                    1592
HasOffers Mobile AppTracking v1.0                                                                                                                                                                                                                                               22
Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1                                      

In [ ]:
len(installs["session_user_agent"].unique())

In [10]:
installs["ref_hash"].isin(clicks["ref_hash"]).value_counts()

False    3405
True        7
Name: ref_hash, dtype: int64

In [ ]:
clicks["agent_device"].value_counts()

In [ ]:
#ver si modelos mas comunes , actividad